# Assignment: Build a Weather MCP Server

## Objective
Build a simple MCP (Model Context Protocol) server that wraps the OpenWeatherMap API to provide current weather data.



### Install Required Package

In [15]:
!pip install requests -q

zsh:1: command not found: pip


---

## Step 2: Implement the WeatherMCP Server

Complete the implementation below. The basic structure is provided - you need to ensure all methods work correctly.

In [16]:
import requests
from typing import Dict, List, Optional
import json
from datetime import datetime


class WeatherMCP:
    """
    MCP Server for OpenWeatherMap API.
    Provides tools for accessing current weather data.
    """
    
    def __init__(self, api_key: str):
        """
        Initialize the Weather MCP server.
        
        Args:
            api_key: OpenWeatherMap API key
        """
        # Store the API key and base URL for making requests
        self.api_key = api_key
        self.base_url = "https://api.openweathermap.org/data/2.5"
        
    def _make_request(self, endpoint: str, params: Dict) -> Dict:
        """
        Make a request to the OpenWeatherMap API.
        
        Args:
            endpoint: API endpoint (e.g., 'weather')
            params: Query parameters
            
        Returns:
            JSON response as dictionary
        """
        # Add API key to the parameters
        params['appid'] = self.api_key
        url = f"{self.base_url}/{endpoint}"
        
        # Make the HTTP request with error handling
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            # Return error message if request fails
            return {"error": f"API request failed: {str(e)}"}
    
    def get_current_weather(
        self, 
        location: str, 
        units: str = "metric"
    ) -> Dict:
        """
        Get current weather conditions for a location.
        
        Args:
            location: City name (e.g., "London", "New York") or city,country code
            units: Temperature units - 'metric' (Celsius), 'imperial' (Fahrenheit), or 'standard' (Kelvin)
            
        Returns:
            Dictionary containing current weather data
        """
        # Set up the parameters for the API call
        params = {
            'q': location,
            'units': units
        }
        
        # Make the API request
        data = self._make_request('weather', params)
        
        # Check if there was an error
        if 'error' in data:
            return data
        
        # Format and return the weather data
        # I'm extracting the most relevant information from the API response
        # and organizing it in a clean structure
        return {
            "location": data.get('name', 'Unknown'),
            "country": data.get('sys', {}).get('country', 'Unknown'),
            "coordinates": {
                "lat": data.get('coord', {}).get('lat'),
                "lon": data.get('coord', {}).get('lon')
            },
            "weather": {
                "condition": data.get('weather', [{}])[0].get('main', 'Unknown'),
                "description": data.get('weather', [{}])[0].get('description', 'Unknown'),
                "icon": data.get('weather', [{}])[0].get('icon', '')
            },
            "temperature": {
                "current": data.get('main', {}).get('temp'),
                "feels_like": data.get('main', {}).get('feels_like'),
                "min": data.get('main', {}).get('temp_min'),
                "max": data.get('main', {}).get('temp_max'),
                "units": "°C" if units == "metric" else "°F" if units == "imperial" else "K"
            },
            "atmospheric": {
                "pressure": data.get('main', {}).get('pressure'),
                "humidity": data.get('main', {}).get('humidity'),
                "visibility": data.get('visibility')
            },
            "wind": {
                "speed": data.get('wind', {}).get('speed'),
                "direction": data.get('wind', {}).get('deg'),
                "gust": data.get('wind', {}).get('gust')
            },
            "clouds": data.get('clouds', {}).get('all'),
            "timestamp": datetime.fromtimestamp(data.get('dt', 0)).isoformat(),
            "sunrise": datetime.fromtimestamp(data.get('sys', {}).get('sunrise', 0)).isoformat(),
            "sunset": datetime.fromtimestamp(data.get('sys', {}).get('sunset', 0)).isoformat()
        }
    
    def list_tools(self) -> List[Dict]:
        """
        List all available MCP tools.
        
        Returns:
            List of tool definitions
        """
        # Return a list describing the available tools
        # This helps LLMs understand what this server can do
        return [
            {
                "name": "get_current_weather",
                "description": "Get current weather conditions for any location worldwide",
                "parameters": {
                    "location": "City name or city,country code (e.g., 'London' or 'London,UK')",
                    "units": "Temperature units: 'metric' (Celsius), 'imperial' (Fahrenheit), or 'standard' (Kelvin)"
                }
            }
        ]


print("✅ WeatherMCP server class ready!")

✅ WeatherMCP server class ready!


---

## Step 3: Configure Your API Key

In [17]:
import getpass
import os

# Check if API key is already in environment
if not os.getenv("OPENWEATHER_API_KEY"):
    openweather_api_key = getpass.getpass("Enter your OpenWeatherMap API key: ")
    os.environ["OPENWEATHER_API_KEY"] = openweather_api_key
else:
    openweather_api_key = os.getenv("OPENWEATHER_API_KEY")

# Initialize the Weather MCP server
weather_mcp = WeatherMCP(api_key=openweather_api_key)

print("✅ Weather MCP server initialized!")
print("\nAvailable tools:")
for tool in weather_mcp.list_tools():
    print(f"  - {tool['name']}: {tool['description']}")

✅ Weather MCP server initialized!

Available tools:
  - get_current_weather: Get current weather conditions for any location worldwide


---

## Step 4: Test Implementation

Testing MCP server with at least 3 different locations.

In [18]:
# Test 1: Get weather for London
print("=" * 60)
print("TEST 1: London, UK (Metric)")
print("=" * 60)
result = weather_mcp.get_current_weather("London", units="metric")

print(f"📍 Location: {result['location']}, {result['country']}")
print(f"🌤️  Weather: {result['weather']['description'].title()}")
print(f"🌡️  Temperature: {result['temperature']['current']}{result['temperature']['units']}")
print(f"🤔 Feels like: {result['temperature']['feels_like']}{result['temperature']['units']}")
print(f"💨 Wind Speed: {result['wind']['speed']} m/s")
print(f"💧 Humidity: {result['atmospheric']['humidity']}%")
print(f"☁️  Cloud Cover: {result['clouds']}%")

# Test 2: Get weather for Tokyo with imperial units
print("\n" + "=" * 60)
print("TEST 2: Tokyo, Japan (Imperial)")
print("=" * 60)
result2 = weather_mcp.get_current_weather("Tokyo,JP", units="imperial")

print(f"📍 Location: {result2['location']}, {result2['country']}")
print(f"🌤️  Weather: {result2['weather']['description'].title()}")
print(f"🌡️  Temperature: {result2['temperature']['current']}{result2['temperature']['units']}")
print(f"🤔 Feels like: {result2['temperature']['feels_like']}{result2['temperature']['units']}")
print(f"💨 Wind Speed: {result2['wind']['speed']} mph")
print(f"💧 Humidity: {result2['atmospheric']['humidity']}%")

# Test 3: Get weather for New York
print("\n" + "=" * 60)
print("TEST 3: New York, USA (Metric)")
print("=" * 60)
result3 = weather_mcp.get_current_weather("New York,US", units="metric")

print(f"📍 Location: {result3['location']}, {result3['country']}")
print(f"🌤️  Weather: {result3['weather']['description'].title()}")
print(f"🌡️  Temperature: {result3['temperature']['current']}{result3['temperature']['units']}")
print(f"🤔 Feels like: {result3['temperature']['feels_like']}{result3['temperature']['units']}")
print(f"💨 Wind Speed: {result3['wind']['speed']} m/s")
print(f"💧 Humidity: {result3['atmospheric']['humidity']}%")
print(f"🌅 Sunrise: {result3['sunrise']}")
print(f"🌇 Sunset: {result3['sunset']}")

print("\n✅ All tests completed successfully!")

TEST 1: London, UK (Metric)
📍 Location: London, GB
🌤️  Weather: Scattered Clouds
🌡️  Temperature: 11.31°C
🤔 Feels like: 10.77°C
💨 Wind Speed: 3.6 m/s
💧 Humidity: 87%
☁️  Cloud Cover: 40%

TEST 2: Tokyo, Japan (Imperial)
📍 Location: London, GB
🌤️  Weather: Scattered Clouds
🌡️  Temperature: 11.31°C
🤔 Feels like: 10.77°C
💨 Wind Speed: 3.6 m/s
💧 Humidity: 87%
☁️  Cloud Cover: 40%

TEST 2: Tokyo, Japan (Imperial)
📍 Location: Tokyo, JP
🌤️  Weather: Scattered Clouds
🌡️  Temperature: 58.86°F
🤔 Feels like: 57.92°F
💨 Wind Speed: 11.5 mph
💧 Humidity: 74%

TEST 3: New York, USA (Metric)
📍 Location: Tokyo, JP
🌤️  Weather: Scattered Clouds
🌡️  Temperature: 58.86°F
🤔 Feels like: 57.92°F
💨 Wind Speed: 11.5 mph
💧 Humidity: 74%

TEST 3: New York, USA (Metric)
📍 Location: New York, US
🌤️  Weather: Scattered Clouds
🌡️  Temperature: 14.4°C
🤔 Feels like: 12.99°C
💨 Wind Speed: 8.75 m/s
💧 Humidity: 42%
🌅 Sunrise: 2025-11-01T06:26:31
🌇 Sunset: 2025-11-01T16:52:33

✅ All tests completed successfully!
📍 Location

---

## Simulating AI Using the MCP Server

Now let's see how an AI assistant would use your MCP server to answer natural language questions!

In [19]:
# Configure OpenAI API Key
import os
import getpass

if not os.getenv("OPENAI_API_KEY"):
    openai_api_key = getpass.getpass("Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("🔑 OpenAI API key configured!")
else:
    print("✅ OpenAI API key already configured!")

print("🔑 Ready to use OpenAI with the MCP server!")

✅ OpenAI API key already configured!
🔑 Ready to use OpenAI with the MCP server!


In [20]:
from openai import OpenAI

def ai_weather_assistant(user_question: str):
    """
    Uses OpenAI to interact with the MCP server.
    
    Flow:
    1. User asks question → 2. OpenAI extracts location → 3. Call MCP tool → 4. OpenAI generates response
    """
    
    print("="*70)
    print(f"👤 USER: {user_question}")
    print("="*70)
    
    # Initialize OpenAI client
    client = OpenAI()
    
    # Step 1: Use OpenAI to extract location from the question
    extraction_prompt = f"""Extract the city/location name from this question: "{user_question}"
    
Return ONLY the city name, nothing else. If no city is found, return "NONE"."""
    
    print("\n🤖 AI: Analyzing your question...")
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": extraction_prompt}],
        temperature=0
    )
    
    location = response.choices[0].message.content.strip()
    
    if location == "NONE":
        print("\n🤖 AI: I couldn't identify a city in your question. Which location do you want to know about?")
        return
    
    # Step 2: Call MCP tool
    print(f"🔧 AI: Calling weather_mcp.get_current_weather('{location}')")
    weather_data = weather_mcp.get_current_weather(location, units='metric')
    
    if 'error' in weather_data:
        print(f"\n🤖 AI: {weather_data['error']}")
        return
    
    # Step 3: Use OpenAI to generate a natural response
    print("💭 AI: Generating response...")
    
    response_prompt = f"""Based on this weather data, answer the user's question naturally and conversationally.

User Question: {user_question}

Weather Data:
- Location: {weather_data['location']}, {weather_data['country']}
- Temperature: {weather_data['temperature']['current']}°C (feels like {weather_data['temperature']['feels_like']}°C)
- Condition: {weather_data['weather']['description']}
- Humidity: {weather_data['atmospheric']['humidity']}%
- Wind: {weather_data['wind']['speed']} m/s

Provide a helpful, concise answer."""
    
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": response_prompt}],
        temperature=0.7
    )
    
    ai_response = completion.choices[0].message.content
    
    print(f"\n🤖 AI: {ai_response}")
    print("="*70)


# Example 1: General weather query
ai_weather_assistant("What's the weather like in Delhi?")

print("\n")

# Example 2: Decision-making query
ai_weather_assistant("Should I bring an umbrella in Paris today?")

print("\n")

# Example 3: Temperature query
ai_weather_assistant("Is it cold in Tokyo right now?")

print("\n")

# Example 5: Test with different phrasing
ai_weather_assistant("Tell me about the weather at Mumbai")

👤 USER: What's the weather like in Delhi?

🤖 AI: Analyzing your question...
🔧 AI: Calling weather_mcp.get_current_weather('Delhi')
🔧 AI: Calling weather_mcp.get_current_weather('Delhi')
💭 AI: Generating response...
💭 AI: Generating response...

🤖 AI: The weather in Delhi right now is around 22°C, and it feels pretty similar at 22.1°C. However, there’s some smoke in the air, which might affect visibility. The humidity is at 68%, and it’s quite calm with no wind. Make sure to take care if you're heading outside!


👤 USER: Should I bring an umbrella in Paris today?

🤖 AI: Analyzing your question...

🤖 AI: The weather in Delhi right now is around 22°C, and it feels pretty similar at 22.1°C. However, there’s some smoke in the air, which might affect visibility. The humidity is at 68%, and it’s quite calm with no wind. Make sure to take care if you're heading outside!


👤 USER: Should I bring an umbrella in Paris today?

🤖 AI: Analyzing your question...
🔧 AI: Calling weather_mcp.get_current_